In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

For sparse optical flow, draw the tracked keypoints onto each frame and try to show the frames fast enough, such that it looks like a video.

In [2]:
## SPARSE OPTICAL FLOW ##
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

ret, frame_new = cap.read()
frame_prev_gray = cv2.cvtColor(frame_new, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    ret, frame_new = cap.read()
    if not ret:
        print("Jo, try again...")

    frame_new_gray = cv2.cvtColor(frame_new, cv2.COLOR_BGR2GRAY)

    feat1 = cv2.goodFeaturesToTrack(frame_new_gray, maxCorners=500, qualityLevel=0.3, minDistance=7)
    feat2, status, error = cv2.calcOpticalFlowPyrLK(frame_new_gray, frame_prev_gray, feat1, None)

    frame_prev_gray = frame_new_gray

    for i in range(len(feat1)):
        f10=int(feat1[i][0][0])
        f11=int(feat1[i][0][1])
        f20=int(feat2[i][0][0])
        f21=int(feat2[i][0][1])
        dist = np.sqrt((f21-f11)**2+(f20-f10)**2)
    
        if dist > 1.5:
            cv2.line(frame_new, (f10,f11), (f20, f21), (0, 255, 0), 2)
            cv2.circle(frame_new, (f10, f11), 5, (0, 255, 0), -1)

    cv2.imshow('Sparse Optical Flow', frame_new)
    # cv2.waitKey(20)

    if cv2.waitKey(1) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()


For dense optical flow, represent the movement in any way you see fitting. For example by making a new image with the colors of each pixel representing the movement.

In [3]:
## DENSE OPTICAL FLOW ##
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

ret, frame_new = cap.read()
frame_prev = frame_new
frame_prev_gray = cv2.cvtColor(frame_new, cv2.COLOR_BGR2GRAY)
flow = None
while cap.isOpened():
    ret, frame_new = cap.read()
    if not ret:
        print("Jo, try again...")

    frame_new_gray = cv2.cvtColor(frame_new, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(frame_prev_gray, frame_new_gray, flow, 0.5, 3, 15, 3, 5, 1.5, 0)

    frame_prev_gray = frame_new_gray

    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1]) 

    mask = np.zeros_like(frame_new) 
    
    mask[..., 1] = 255
    mask[..., 0] = ang * 180 / np.pi / 2
    mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    final_mask = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    
    cv2.imshow('Dense Optical Flow', final_mask)

    if cv2.waitKey(1) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()

/tmp/ipykernel_8198/4240105984.py:24: RuntimeWarning: invalid value encountered in cast
  mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)


In [4]:
## DENSE OPTICAL FLOW ##
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

ret, frame_new = cap.read()
frame_prev = frame_new
frame_prev_gray = cv2.cvtColor(frame_new, cv2.COLOR_BGR2GRAY)
flow = None
while cap.isOpened():
    ret, frame_new = cap.read()
    if not ret:
        print("Jo, try again...")

    frame_new_gray = cv2.cvtColor(frame_new, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(frame_prev_gray, frame_new_gray, flow, 0.5, 3, 15, 3, 5, 1.5, 0)

    frame_prev_gray = frame_new_gray

    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1]) 

    h, w = frame_prev.shape[:2]
    flow_vis = np.zeros_like(frame_new)
    flow_vis[..., 0] = flow[..., 0] * 10 + 128
    flow_vis[..., 1] = flow[..., 1] * 10 + 128
    flow_vis[..., 2] = 128

    alpha = 0.3
    output = cv2.addWeighted(frame_new, alpha, flow_vis, 1 - alpha, 0)

    cv2.imshow('Dense Optical Flow', output)

    if cv2.waitKey(1) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
# import time
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

# get the ROI
flag, frame = cap.read()
roi = cv2.selectROI(frame, False)
imCrop = frame[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]

x, y, w, h = roi        # w, h will stay the same but x, y will be updated
prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[y:y+h, x:x+w]

cv2.imshow("Image", prev_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()
# time_previous = time.time()
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

flow = None

while True:
    flag, frame = cap.read()
    # dt = time.time() - time_previous
    # time_previous = time.time()

    if not flag:
        print("Jo, try again...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # get flow ONLY of the ROI
    # see flow as partial derivatives in x and y direction -> gradient of image
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray[y:y+h, x:x+w], flow, 0.5, 3, 15, 3, 2, 1.5, 0)
    
    # get the flow from the ROI
    # flow_roi = flow #[y:y+h, x:x+w, :]

    # take the mean in x and y direction and calculate new points

    # x += int(flow[..., 0].max()) if (flow[..., 0].mean() > 0) else int(flow[..., 0].min())
    # y += int(flow[..., 1].max()) if (flow[..., 1].mean() > 0) else int(flow[..., 1].min())
    scaling = 2
    x += int(scaling * flow[..., 0].mean() ) 
    y += int(scaling * flow[..., 1].mean() ) 

    prev_gray = gray[y:y+h, x:x+w]

    # cv2.imshow("Frame", frame)
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    cv2.imshow("Frame with ROI", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [7]:
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

flag, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
corners0 = cv2.goodFeaturesToTrack(prev_gray, maxCorners=500, qualityLevel=0.3, minDistance=7)

mask = np.zeros_like(prev_frame)

while True:
    flag, frame = cap.read()
    if not flag:
        print("Jo, try again...")
        cap.release()
        cv2.destroyAllWindows()
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(corners0.shape)

    # get 2D points of new pos of corners
    corners1, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, corners0, None)
    # st tells us if flow for corresponding feature has been found (1)
    print("Optical Flow")
    print(corners1.shape)
    
    # take only the ones where the corresponding feature was actually found 
    # https://www.geeksforgeeks.org/python-opencv-optical-flow-with-lucas-kanade-method/
    good_new = corners1[status==1]
    good_old = corners0[status==1]


    for i in range(0, good_new.shape[0]):
        f10=int(good_new[i][0])
        f11=int(good_new[i][1])
        f20=int(good_old[i][0])
        f21=int(good_old[i][1])
        dist = np.sqrt((f21-f11)**2+(f20-f10)**2)
    
        if dist > 1.5:
            cv2.line(mask, (f10,f11), (f20, f21), (0, 255, 0), 2)
            cv2.circle(frame, (f10, f11), 5, (255, 0, 0), -1)

    img = cv2.add(frame, mask)

    cv2.imshow("frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

    prev_gray = frame_gray.copy()
    corners0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()

(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(340, 1, 2)
Optical Flow
(340, 1, 2)
(

In [8]:
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Robots.mp4')   

# get the ROI
flag, prev_frame = cap.read()
roi = cv2.selectROI(prev_frame, False)
imCrop = prev_frame[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
imCrop_gray = cv2.cvtColor(imCrop, cv2.COLOR_BGR2GRAY)  # get image with features to track

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
corners0 = cv2.goodFeaturesToTrack(imCrop_gray, maxCorners=500, qualityLevel=0.3, minDistance=7) 

print(corners0.shape)
print(corners0)

corners0[:,0, 0] += int(roi[0])
corners0[:, 0, 1] += int(roi[1])
# threshold crners to only get the ones that are in our ROI
# corners0 = []
# for i in range(len(feat1)):
#     x=int(feat1[i][0][0])
#     y=int(feat1[i][0][1])
#     if(x >= int(roi[1]) and x <= int(roi[1]+roi[3]) and y >=int(roi[0]) and y <= int(roi[0]+roi[2])):
#         corners0.insert()
# corners0 = 

mask = np.zeros_like(prev_frame)

while True:
    flag, frame = cap.read()
    if not flag:
        print("Jo, try again...")
        cap.release()
        cv2.destroyAllWindows()
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(corners0.shape)

    # get 2D points of new pos of corners
    corners1, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, corners0, None)
    # st tells us if flow for corresponding feature has been found (1)
    print("Optical Flow")
    print(corners1.shape)
    
    # take only the ones where the corresponding feature was actually found 
    # https://www.geeksforgeeks.org/python-opencv-optical-flow-with-lucas-kanade-method/
    good_new = corners1[status==1]
    good_old = corners0[status==1]


    for i in range(0, good_new.shape[0]):
        f10=int(good_new[i][0])
        f11=int(good_new[i][1])
        f20=int(good_old[i][0])
        f21=int(good_old[i][1])
        dist = np.sqrt((f21-f11)**2+(f20-f10)**2)
    
        if dist > 1.5:
            cv2.line(mask, (f10,f11), (f20, f21), (0, 255, 0), 2)
            cv2.circle(frame, (f10, f11), 5, (255, 0, 0), -1)

    img = cv2.add(frame, mask)

    cv2.imshow("frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

    prev_gray = frame_gray.copy()
    corners0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
(90, 1, 2)
[[[160. 132.]]

 [[129. 262.]]

 [[165.  56.]]

 [[117.  15.]]

 [[ 71.  94.]]

 [[123. 101.]]

 [[179. 238.]]

 [[188. 204.]]

 [[127.  93.]]

 [[123. 125.]]

 [[145. 144.]]

 [[116.  54.]]

 [[122. 108.]]

 [[182. 224.]]

 [[ 59. 209.]]

 [[144.  74.]]

 [[152. 138.]]

 [[ 61. 184.]]

 [[175.  25.]]

 [[ 44. 184.]]

 [[153. 123.]]

 [[ 39. 208.]]

 [[142.  11.]]

 [[158.  74.]]

 [[ 98. 230.]]

 [[116.  47.]]

 [[ 47. 107.]]

 [[ 39. 243.]]

 [[164. 176.]]

 [[157. 177.]]

 [[ 66. 215.]]

 [[184. 190.]]

 [[ 76. 259.]]

 [[ 37. 201.]]

 [[101. 177.]]

 [[146. 115.]]

 [[ 34. 187.]]

 [[ 69. 107.]]

 [[181. 102.]]

 [[146. 299.]]

 [[146. 178.]]

 [[ 57. 236.]]

 [[123. 164.]]

 [[177.  14.]]

 [[107. 186.]]

 [[107. 173.]]

 [[144. 291.]]

 [[132. 180.]]

 [[124. 144.]]

 [[109. 118.]]

 [[183. 247.]]

 [[108. 111.]]

 [[ 75. 244.]]

 [[176. 144.]]

 [[112. 181.]]

 [[170.

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
cap = cv2.VideoCapture('/home/hhauter/Documents/S23/Perception/34759-Perception-Exercises/Project_week_2/Challenge.mp4')   

# get the ROI
flag, prev_frame = cap.read()
roi = cv2.selectROI(prev_frame, False)
imCrop = prev_frame[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
imCrop_gray = cv2.cvtColor(imCrop, cv2.COLOR_BGR2GRAY)  # get image with features to track

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(imCrop_gray, None)
cv2.destroyAllWindows()



mask = np.zeros_like(prev_frame)
bf = cv2.BFMatcher()
while True:
    flag, frame = cap.read()
    if not flag:
        print("Jo, try again...")
        cap.release()
        cv2.destroyAllWindows()
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    kp2, des2 = sift.detectAndCompute(frame_gray, None)

    matches = bf.match(des1, des2)

    # get 2D points of new pos of corners

    
    matches = sorted(matches, key = lambda x:x.distance)


    img3 = cv2.drawMatches(imCrop_gray,kp1,frame,kp2,matches[:50],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    cv2.imshow("frame", img3)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

    prev_gray = frame_gray.copy()

cap.release()
cv2.destroyAllWindows()

# how to improve this 

# follow robot (bounding box, kalman etc.) and take new images every x frames and use that one for feature matching 
# pre process image of robot so that it is isolated
# compute distance of points in prev frame and current frame and threshold them 


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
